In [97]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation as cv
from xgboost.sklearn import XGBClassifier
import matplotlib.pyplot as plt
%matplotlib inline

In [98]:
#不要なPassengerId,Cabinを削除。カテゴリー変数を数値に。Embarkedの穴を埋める。目的変数を取り出す。
df = pd.read_csv("train.csv")
y = df["Survived"].as_matrix()
df = df.drop(["PassengerId","Cabin"], axis = 1)
df = df.replace("male",0).replace("female",1).replace("S",0).replace("C",1).replace("Q",2)
df = df.fillna(value = {'Embarked': 1})

#敬称ごとに変数を追加
honor = []
for i in df["Name"]:
    if "Master" in i :
        honor.append(int(0))
    elif "Mr" in i :
        honor.append(int(1))
    elif "Miss" in i :
        honor.append(int(2))
    elif "Mrs" in i :
        honor.append(int(3))
    else:
        honor.append(int(4))
df['honor'] = np.array(honor)

#家族の人数を追加
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1

#年齢の欠損を敬称別の中央値で埋める。
df = df.fillna(value = {"Age":-1})
a = df.loc[:,["Age","honor"]]
a =np.array(a)
for i in a:
    if i[0] == -1:
        i[0] =df[df["honor"]==i[1]]["Age"].median()
frame = pd.DataFrame(a, columns=["Age", "honor"])
df = df.drop(["Age","honor"], axis = 1)
df = pd.concat([df, frame], axis=1)

In [99]:
#家族別の生存率を追加
name_df_train = df.iloc[:,2]
survive_array = df.iloc[:,0]

name_df = name_df_train

name_array = []
name_count = []
name_survive_count = []
name_survive_ratio = []

name_i = 0

for name in name_df:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')
    if name_split_first in name_array:
        name_count[name_array.index(name_split_first)] = name_count[name_array.index(name_split_first)] + 1
        name_survive_count[name_array.index(name_split_first)] = name_survive_count[name_array.index(name_split_first)] + survive_array[name_i]                            
    else:
        name_array.append(name_split_first)
        name_count.append(1)
        name_survive_count.append(survive_array[name_i])
    name_i = name_i + 1
        
count = 0
for i in name_count:
    if name_count[count] == 1:
        name_survive_ratio.append(0)
    else:
        name_survive_ratio.append(name_survive_count[count]/(name_count[count]))
    count = count + 1
    
survive_ratio_array = []    

for name in name_df:
    name_split = name.split(" ")
    name_split_first = name_split[0].replace(',','')
    survive_ratio_array.append(name_survive_ratio[name_array.index(name_split_first)])

df['familysurviveratio'] = np.array(survive_ratio_array)

In [100]:
df.head()

,Survived,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Embarked,FamilySize,Age,honor,familysurviveratio
0,0,3,"Braund, Mr. Owen Harris",0,1,0,A/5 21171,7.2500,0.0,2,22.0,1.0,0.0
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,1,0,PC 17599,71.2833,1.0,2,38.0,1.0,0.0
2,1,3,"Heikkinen, Miss. Laina",1,0,0,STON/O2. 3101282,7.9250,0.0,1,26.0,2.0,0.0
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,1,0,113803,53.1000,0.0,2,35.0,1.0,0.5
4,0,3,"Allen, Mr. William Henry",0,0,0,373450,8.0500,0.0,1,35.0,1.0,0.5


In [101]:
def name_classifier(name_df):    
    name_class_df = pd.DataFrame(columns={'miss','mrs','master','mr'})
    
    for name in name_df:        
        if 'Miss' in name:
            df = pd.DataFrame([[1,0,0,0]],columns={'miss','mrs','master','mr'})
        elif 'Mrs' in name:
            df = pd.DataFrame([[0,1,0,0]],columns={'miss','mrs','master','mr'})
        elif 'Master' in name:
            df = pd.DataFrame([[0,0,1,0]],columns={'miss','mrs','master','mr'})
        elif 'Mr' in name:
            df = pd.DataFrame([[0,0,0,1]],columns={'miss','mrs','master','mr'})
        else :
            df = pd.DataFrame([[0,0,0,0]],columns={'miss','mrs','master','mr'})
        name_class_df = name_class_df.append(df,ignore_index=True)        
    return name_class_df

In [102]:
name_classifier(df.Name)
df = pd.concat([df,name_classifier(df.Name) ], axis=1)
df = df.drop(["honor","Sex","SibSp","Parch","Ticket"],axis= 1)

In [103]:
df = df.drop(["Survived","Name","Fare"],axis=1)

In [104]:
df = df.drop(["Embarked"],axis=1)


In [105]:
df[:10]

,Pclass,FamilySize,Age,familysurviveratio,miss,master,mr,mrs
0,3,2,22.0,0.000000,0,0,0,1
1,1,2,38.0,0.000000,0,1,0,0
2,3,1,26.0,0.000000,1,0,0,0
3,1,2,35.0,0.500000,0,1,0,0
4,3,1,35.0,0.500000,0,0,0,1
5,3,1,27.0,0.333333,0,0,0,1
6,1,1,54.0,0.000000,0,0,0,1
7,3,5,2.0,0.000000,0,0,1,0
8,3,3,27.0,0.500000,0,1,0,0
9,2,2,14.0,0.500000,0,1,0,0


In [106]:
X = df.as_matrix()

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [123]:
mod = xgb.XGBClassifier()
mod.fit(X_train, y_train)
print('train accuracy: %.3f' % mod.score(X_train, y_train))
print('test  accuracy: %.3f' % mod.score(X_test, y_test))

train accuracy: 0.881
test  accuracy: 0.869


In [124]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(min_samples_leaf=3, random_state=0)
forest.fit(X_train, y_train)
print('Train score: {}'.format(forest.score(X_train, y_train)))
print('Test score: {}'.format(forest.score(X_test, y_test)))

Train score: 0.8683788121990369
Test score: 0.8619402985074627
